In [2]:
''' 
 Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
 Date; 9. 24. 2021 - 11. 13. 2021
 Title: Artificial Intelligence_Project 2
 Professor: Seung-Hoon Na
 Reference: https://cpm0722.github.io/pytorch-implementation/transformer
            https://pytorch.org/tutorials/beginner/transformer_tutorial.html'''

' \n Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering\n Date; 7. . 2021 - 2. 5. 2021\n Title: Artificial Intelligence_Project 2\n Professor: Seung-Hoon Na'

In [4]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# Encoder
 - Generate context (vector made by Encoding)
 - compress all of the information in sentences without leakage
 - pad mask: do not assign attention to pad token

# Decoder
- make a sentence from sentence, context
- INPUT: context(output of Encoder), some sentence(Ground Truth[:-1]'s sentence)
- subsequent mask: for hiding token's solution (future word)

# Teacher Forcing: use labeled data(Ground Truth) as input in Supervised Learning

# Self-Attention
- 'Self': find attention in a same sentence
- Attention: guess what a word(ex. it or the thrid person singular) points in a sentecne
- Query: now token
- Key: target token to find attention
- Value: same Key

In [5]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout = 0.5):
        
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout) # include information of word's location
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)
        
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform(-initrange, initrange)
        
    def forward(self, src, src_mask):
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal = 1)

# Transformer’s Input (Positional Encoding)
- purpose of positional encoding: normalize positional information -> limit specific range(even index: sin, odd indxe: cos)

# Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p = dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Load and batch data

In [12]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))
'''
torch.numel: Returns the total number of elements in the input tensor.
filter(function, iterable): 
torch.cat(tensor, dim, out):  
'''
# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
'''
train_iter = WikiText2(split='train')
val_iter = WikiText2(split='val')
test_iter = WikiText2(split='test')
'''
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def batchify(data: Tensor, bsz: int) -> Tensor:
#     """Divides the data into bsz separate sequences, removing extra elements
#     that wouldn't cleanly fit.

#     Args:
#         data: Tensor, shape [N]
#         bsz: int, batch size

#     Returns:
#         Tensor of shape [N // bsz, bsz]
#     """
#     seq_len = data.size(0) // bsz
#     data = data[:seq_len * bsz]
#     data = data.view(bsz, seq_len).t().contiguous()
#     return data.to(device)

# batch_size = 20
# eval_batch_size = 10
# train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
# val_data = batchify(val_data, eval_batch_size)
# test_data = batchify(test_data, eval_batch_size)

# Conclusion: vocab = word embedding

In [ ]:
train_iter = WikiText2(split='train') # load data
tokenizer = get_tokenizer('basic_english') # make tokens
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>']) 
'''
 map(fucntion, iterable(list, tuple etc))
 conduct function until iterable 
'''
vocab.set_default_index(vocab['<unk>'])

# Make data

In [ ]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))
'''
torch.numel: Returns the total number of elements in the input tensor.
filter(function, iterable): 
torch.cat(tensor, dim, out):  
'''
# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
'''
train_iter = WikiText2(split='train')
val_iter = WikiText2(split='val')
test_iter = WikiText2(split='test')
'''
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

In [13]:
next(train_iter)

StopIteration: 